# Survey counts

This notebook counts the number of surveys in each electricity access category and compares to the total number of households in each category.

- Total Households from AE data

## TODO

- Use `.any` and `.all` to verify valid surveys.
- Make more appropriate labels for tables
- Are these surveys valid across all the columns we want?
- Needs general cleaning up for readability
- Extend this method to get valid answers for usage and hours

In [1]:
import pandas as pd
import pysentani as sti
survey = pd.read_excel('../data-survey/sentani-merged-cleaned-2015-06-10.xlsx')
survey['access_type'] = sti.access_type(survey)

We consider a household survey valid in these statistics if that household answered the questions about appliance ownership that we are analyzing.

Specifically, TV, radio, refrigerator, fan, rice cookers, and electric lighting.

Valid Survey: answered questions on appliance ownership.

AE count: number of total households according to Advancing Energy.

Valid fraction: valid surveys as a percentage of total households in AE count.

- $N_{h}$ is the total number of households present in the area of interest
- $N_{vos}$ is the number of surveys with valid appliance ownership responses
- $N_{vts}$ is the number of surveys with valid appliance time usage responses

We find $N_{vos}$ by 

- creating a subset of the survey with our appliance ownership questions
- looking for how many of these surveys have incomplete appliance ownership questions
- verifying that surveys either answer all appliance ownership questions or no appliance ownership questions
- are there patterns to the non-responses that would influence the generalizability of the results?

Notes:

- There are examples where app_now and app_buy are not both answered.
- The mobile phone question (`HP_y_n`) was in a separate survey block and has different response patterns.


Below we verify that respondents answered either all or none of the appliance ownership questions.

In [2]:
columns = ['app_now/TV', 
           'app_now/radio', 
           'app_now/fridge', 
           'app_now/fan',
           'app_now/rice_cooker',
           'app_now/lighting']

anynulls = survey[columns].isnull().any(axis=1)
allnulls = survey[columns].isnull().all(axis=1)

# if any and all are the same series, then there are no surveys with some nulls
print('any and all nulls equal?', anynulls.equals(allnulls))

anynulls.value_counts()

any and all nulls equal? True


False    1035
True      149
dtype: int64

In [4]:
# i want to use notnull and all and a group by to get the valid survey counts
survey['valid'] = survey[columns].notnull().all(axis=1)
#subsurvey.groupby('access_type').count()

In [5]:
survey.groupby('access_type').sum()['valid']

access_type
PLN_grid               568.0
PLN_microgrid          158.0
community_microgrid     52.0
no_access              257.0
Name: valid, dtype: float64

In [6]:
survey.groupby('access_type').sum()['valid'].sum()

1035.0

Below we create a table with the total data on households and valid household surveys.

In [7]:
# TODO: come up with a way that uses all columns as above to filter 

appliance_data = survey[['app_now/TV', 'access_type']]
appliance_data_no_nulls = appliance_data.dropna()
adnn = appliance_data_no_nulls.groupby('access_type').count()
adnn['valid_surveys'] = adnn['app_now/TV']

# add value counts column to count of valid surveys
adnn['total_surveys'] = appliance_data['access_type'].value_counts()
adnn['percent_valid'] = adnn['valid_surveys'] / adnn['total_surveys']
# this data from AE counts and Joshua data
adnn['AE_count'] = pd.Series({'PLN_grid':660, 'PLN_microgrid':170, 'community_microgrid':60, 'no_access':398})
adnn['valid_fraction_HH'] = adnn['valid_surveys'] / adnn['AE_count']
adnn['percent_surveyed'] = adnn['total_surveys'] / adnn['AE_count']

output_columns = ['total_surveys', 'AE_count', 'percent_surveyed', 'valid_fraction_HH']

adnn[output_columns]

,total_surveys,AE_count,percent_surveyed,valid_fraction_HH
access_type,,,,
PLN_grid,619,660,0.937879,0.860606
PLN_microgrid,170,170,1.000000,0.929412
community_microgrid,54,60,0.900000,0.866667
no_access,341,398,0.856784,0.645729


In [8]:
# TODO: is there a way to directly include this in the manuscript?
import tabulate
output = adnn[output_columns]
print(tabulate.tabulate(output, headers=output_columns, tablefmt='pipe'))

|                     |   total_surveys |   AE_count |   percent_surveyed |   valid_fraction_HH |
|:--------------------|----------------:|-----------:|-------------------:|--------------------:|
| PLN_grid            |             619 |        660 |           0.937879 |            0.860606 |
| PLN_microgrid       |             170 |        170 |           1        |            0.929412 |
| community_microgrid |              54 |         60 |           0.9      |            0.866667 |
| no_access           |             341 |        398 |           0.856784 |            0.645729 |
